# PDEs and their solutions

General partial differential equation I would like to solve:
$$
G\Big( \vec{x}, \; f(\vec{x}), \;  \partial_{i}f(\vec{x}), \; \partial_{i}\partial_{j}f(\vec{x}) \Big) = 0
$$
for $\vec{x} \in \Omega$ (i.e. some $n$-dimentional domain).

In order for this to have a solution, the following must be true (the *Cauchy–Kowalevski theorem*):

1.  We should be able to solve for the highest derivative. That is, there should exist an analytic $F$ so that 
$$
\partial_k\partial_k f(\vec{x})=F\Big(\vec{x}, f(\vec{x}), \partial_i f(\vec{x}), 
\left\{ \partial_i \partial_j f(\vec{x}) \right\}_{ij \neq kk} \Big)
$$

2. We should know:
$$
f(\vec{x})\Big|_{x_{k}=x_{k}^{(0)}} = L_{0}\Big(\left\{ x_i \right\}_{i \neq k}\Big)
\\
\partial_k f(\vec{x})\Big|_{x_{k}=x_{k}^{(0)}} = L_{1}\Big(\left\{ x_i \right\}_{i \neq k}\Big) 
$$

## Usual PDE problems

In general proving that the given conditions give unique well define solution is non-trivial. However, usually, PDEs are given with some *boundary conditions* of the form
$$
H\Big(\vec{x},f(\vec{x}),\partial_i f(\vec{x}) \Big)\Big|_{\vec{x} \in {\bf S} } =0 \;,
$$
with ${\bf S}$ the boundary of the region in which we look for a solution.

## First order PDEs

In this notebook, we will try to solve general first order PDE inside the $n$-dimensional box. We will assume that there are boundary conditions of the form
$$
H\Big(\vec{x},f(\vec{x}),\partial_i f(\vec{x}) \Big)\Big|_{\vec{x} \in {\bf S} } =0 \;, 
$$
because it will be easier to generalize to 2nd order PDEs later. Probably I will have to make an object called ```Boundary```that generate points inside and on the boundary,  then I can demand that $H=0$ on the boundary points. Let's see how it goes.

# Example

As we build the code, let's try to solve the first order PDE
$$
\dfrac{\partial f}{\partial x} +k \dfrac{\partial f}{\partial y} = x
$$
in $x,y \in [0,1]$ ($2$-dimensional box).

Note that for such PDE we need 

1. An analytic function ($F$) such that $\dfrac{\partial f}{\partial x} = F\left(x,y,f, \dfrac{\partial f}{\partial y}  \right)$ or 
$\dfrac{\partial f}{\partial y} = F\left(x,y,f, \dfrac{\partial f}{\partial x}  \right)$  (which obviously holds).

2. $f(x=x_0,y)=h(y)$ or $f(x,y=y_0)=g(x)$.


The solution is 
$$
f(x,y)=\dfrac{1}{2}x^2 + (y-k \ x) \; c
$$

Note that the condition $f(x=x_0,y)=h(y)$ is not a boundary condition, but I still call it this, because it does not affect the code, and it will be helpful when we generalize it.


In [1]:
import numpy as np


import matplotlib
matplotlib.use('nbAgg')
import matplotlib.pyplot as plt

This is how a general function looks-like. It depends on various free parameters (${\bf w}$) that can be adjusted in order to solve the PDE.

We will use $k=\pi $ and $f(0,y)=\dfrac{y}{10}$, and write the solution as
$f(x,y)=w_0 x^2 + w_1 (y-w_2 \ x)$. So the solution is given for

$$
w_0=\dfrac{1}{2}\\
w_1=\dfrac{1}{10}\\
w_2=\pi
$$

Example guess solution.
In genearl, you can change ```__call__```, or pass it as argument, For the moment it is fine.





In [2]:
class Model:
    def __init__(self,w0,dim_w,dim_x):
        self.w=w0
        self.dim_w=dim_w
        self.dim_x=dim_x
        
        
        self.dfdx=[0 for _1 in range(self.dim_x)]
        
        
    def __call__(self,x):
        return self.w[0]*x[0]**2 + self.w[1]*(x[1]-self.w[2]*x[0])
    
    def derivative(self,x,h=1e-5):
        
        f0=0
        f1=0
        for i in range(self.dim_x):
            x[i]+=h
            f1=self(x)
            
            x[i]+=-2*h
            f0=self(x)
            
            self.dfdx[i]=(f1-f0)/(2*h)



In [3]:
guess=Model(w0=[1,1,2],
                dim_w=3,
                dim_x=2
               )

In [4]:
guess([10,33])

113

In [5]:
guess.derivative([1,1])

In [6]:
guess.dfdx

[0.0, 1.000000000001]

# The boundary conditions

Boundary conditions is given as in the class below. It seems to be convinient (and easy to generalize to more dimensions and more complicated boundaries) to have a function that takes a point ($\vec{x}$) and returns a projection of this point on the boundary ($\vec{x}_B$), then take $H(\vec{x},f,\partial_i f)\Big|_{\vec{x}=\vec{x}_B} = 0$. 


Also, it would be convinient to define a function that returns a random point inside the boundary, which will be used to generate points that will be used to train the model.

So, we can do something like this

In [7]:
class Boundary:
    def __init__(self,model):
        self.model=model
        
        
        #zero for everything, and non-zero for the condition you want for this example 
        self.BC=[[lambda x:0,lambda x:0] for _ in range(model.dim_x)]
        self.BC[0][0]=lambda x: model([0,x[1]]) - 0.1*x[1]
        
    def randomPoint(self):
        
        return [np.random.rand() for _ in range(self.model.dim_x)]
    
    def boundaryPoint(self,x):
        '''
        Note that in this cace y can be \pm \infty, but I choose y \in [0,1]
        '''
        xB=x[:]
        xB[0]=0
        return xB
        
    def randomBoundaryPoint(self):
        xB=self.randomPoint()
        xB[0]=0
        return xB
 
    def boundaryCondition(self,x):
        
        xB=self.boundaryPoint(x)
    
        return self.model(xB) - 0.1*xB[1]
    
    def randomBoundaryCondition(self):
        return self.boundaryCondition(self.randomPoint())

In [8]:
S=Boundary(guess)

In [9]:
[S.boundaryCondition([555,2]),
(S.boundaryPoint([5,2]),S.randomPoint(),S.randomBoundaryPoint()),
S.randomBoundaryCondition()]

[1.8,
 ([0, 2], [0.1965543661376924, 0.4033083671383062], [0, 0.23294903759227836]),
 0.6735088872596295]

# The coefficient matrix

The general form of a 1st order PDE can be written as

$$
\sum_{i=0}^{n} A_{i}\left(\vec{x},  f(\vec{x}) \right) \ \partial_i f(\vec{x}) = {\rm RHS}(\vec{x},f(\vec{x})) \;,
$$

with $A_{i}\left(\vec{x},  f(\vec{x}) \right)$ a known coefficient $n$-column, and ${\rm RHS}(\vec{x},f(\vec{x}))$ the right-hand-side of the PDE.


For the case under study, we have (let's take $k=\pi$)

$$
A = \Bigg(\begin{matrix} 1 \\ \pi \end{matrix}\Bigg) \\
{\rm RHS}=x
$$

In [10]:
# I use classes becauseI will generalize it to second order PDEs, and I will have to get the derivatives of the model.

class Coefficient:
    def __init__(self,model):
        self.model=model
        
        self.coeff=[lambda x:0 for _ in range(self.model.dim_x)]
        
        self.coeff[0]=lambda x: 1
        self.coeff[1]=lambda x: np.pi
        
    def __call__(self,x):
        return [c(x) for c in self.coeff]

    
    
class RightHandSide:
    def __init__(self,model):
        self.model=model
        
    def __call__(self,x):
        
        return x[0]

In [11]:
A=Coefficient(guess)
RHS=RightHandSide(guess)

In [12]:
A([1,2])

[1, 3.141592653589793]

In [13]:
RHS([5,1])

5

We can put everything together, and define a class that describes the differential equation

In [14]:
class DifferentialEquation:
    def __init__(self,Model,Coefficient,RightHandSide,Boundary):
        
        self.model=Model
        
        self.A=Coefficient
        
        self.RHS=RightHandSide
        
        self.Boundary=Boundary
        
        
    def LHS(self,x,h=1e-5):
        lhs=0
        self.model.derivative(x,h)
        dfdx=self.model.dfdx
        A=self.A(x)
        for i  in range(self.model.dim_x):
            lhs+=A[i]*dfdx[i]
        
        return lhs
    
    
    
    def loss(self,x):
        # loss at x
        Qx=(self.LHS(x)-self.RHS(x))**2
        return Qx
        
    def lossBC(self,x):        
        # Mean loss at the boundary. 
        return self.Boundary.boundaryCondition(x)**2
    
    def averageLoss(self,x):
        # averaged loss (loss at the boundary + loss at x)/2 
        return (self.lossBC(x) + self.loss(x))/2.
    
    def randomAverageLoss():
        #average loss at random points
        x=Boundary.randomPoint()
        xB=Boundary.randomBoundaryPoint()
        
        return (self.loss(x)+self.lossBC(xB))/2.
    
    def lossGrad(self,x,h=1e-3):
        '''
        with the definition of h_eff, h=1e-3 should be enough.
        In any case, the idea is to compute the derivetive analytically in general.
        '''
        grad=[0 for i in  range(self.model.dim_w)]
        
        w=self.model.w[:]
        for dim in range(self.model.dim_w):
            h_eff=h*w[dim]+h
            
            self.model.w[dim]=w[dim]-h_eff
            Q0=self.averageLoss(x)

            self.model.w[dim]=w[dim]+h_eff
            Q1=self.averageLoss(x)
            
            self.model.w[dim]=w[dim]
            

            grad[dim]=(Q1-Q0)/(2.*h_eff)

        return grad
    
    def randomLossGrad(self,h=1e-3):
        '''Get the gradient of the averge loss at random point and a random boundary point'''
        
        x=self.Boundary.randomPoint()
        xB=self.Boundary.randomBoundaryPoint()
        
        grad=[0 for i in  range(self.model.dim_w)]
        
        w=self.model.w[:]
        for dim in range(self.model.dim_w):
            h_eff=h*w[dim]+h
            
            self.model.w[dim]=w[dim]-h_eff
            Q0=(self.loss(x)+self.lossBC(xB))/2.

            self.model.w[dim]=w[dim]+h_eff
            Q1=(self.loss(x)+self.lossBC(xB))/2.
            
            self.model.w[dim]=w[dim]
            

            grad[dim]=(Q1-Q0)/(2.*h_eff)

        return grad


So far this is the definiton of the PDE.

In [15]:
guess=Model(w0=[1,1,2],
                dim_w=3,
                dim_x=2
               )

S=Boundary(guess)

A=Coefficient(guess)
RHS=RightHandSide(guess)

In [16]:
PDE=DifferentialEquation(guess,A,RHS,S)

Check that ${\rm LHS} = 2w_0 x - w_1 w_2 +\pi w_1$ and ${\rm RHS}=x$.

Our current values are $w_0 = 1$, $w_1 = 1$ and $w_2 = 2$. That is ${\rm LHS}=2x-2+\pi$.




In [17]:
#it seems correct
x=3
y=2

PDE.LHS([x,y]),  2*PDE.model.w[0]*x-PDE.model.w[1]*PDE.model.w[2]+np.pi*PDE.model.w[1]

(7.141592653636579, 7.141592653589793)

Check that the loss is 
$$
\left({\rm LHS} - {\rm RHS}\right)^2=\left( 2w_0 x - w_1 w_2 +\pi w_1 -x \right)^2 = 
\left( x - 2 + \pi \right)^2
$$



The loss at the boundary (as defined by our convention before) is:
$$
\left(f(0,y)-\dfrac{1}{10}y\right)^2=\left(w_1 y -\dfrac{1}{10}y\right)^2= y^2 (w_1 - 0.1)^2 
$$


In [18]:
#it seems correct

x=3
y=2

print(PDE.loss([x,y]), (2*PDE.model.w[0]*x-PDE.model.w[1]*PDE.model.w[2]+np.pi*PDE.model.w[1] - x)**2)

print(PDE.lossBC([x,y]), y**2*(0.1-PDE.model.w[1])**2)

17.152872540609547 17.152789708268944
3.24 3.24


The average loss should return the average of te previous two.

In [19]:
#again correct!
PDE.averageLoss([x,y]), (PDE.loss([x,y])+PDE.lossBC([x,y]))/2.

(10.196436270304773, 10.196436270304773)

The gradient of the average loss (defined by our convention) is:
$$
\dfrac{\partial Q}{\partial w_0} = 2x\left( 2w_0 x - w_1 w_2 +\pi w_1 -x \right) \\
\dfrac{\partial Q}{\partial w_1} = (\pi-w_2)\left( 2w_0 x - w_1 w_2 +\pi w_1 -x \right) + y^2 (w_1 - 0.1) \\
\dfrac{\partial Q}{\partial w_2} = -w_1 \left( 2w_0 x - w_1 w_2 +\pi w_1 -x \right)
$$

In [20]:
# PDE.lossGrad([x,y])

print(PDE.lossGrad([x,y]))
[
2*x*(2*PDE.model.w[0]*x-PDE.model.w[1]*PDE.model.w[2]+np.pi*PDE.model.w[1]-x),   
(np.pi - PDE.model.w[2]) * (2*PDE.model.w[0]*x-PDE.model.w[1]*PDE.model.w[2]+np.pi*PDE.model.w[1]-x)+
y**2*(PDE.model.w[1] -0.1),
-PDE.model.w[1]*(2*PDE.model.w[0]*x-PDE.model.w[1]*PDE.model.w[2]+np.pi*PDE.model.w[1]-x)
]

[24.835140466266115, 8.313500299419907, -4.1511601956161215]


[24.84955592153876, 8.328011747499565, -4.141592653589793]

We can now find the minimum of the average loss function, using SGD!

For now, I copy the NAdam from [ASAP](https://github.com/dkaramit/ASAP/tree/master/Optimization/Stochastic-Gradient-Descent/python)

In [21]:
#class for Stochastic Gradient Descent
class StochasticGradientDescent:    
    def __init__(self,strategy):
        self.strategy=strategy
    
    def run(self,abs_tol=1e-5, rel_tol=1e-3, step_break=100,max_step=5000):
        '''        
        abs_tol, rel_tol, step_break: stop when _check<1 (_check is what update should return) 
        for step_break consecutive steps
        
        max_step: maximum number of steps
        '''
        _s=0
        count_steps=1
        while count_steps<=max_step:
            _check=self.strategy.update(abs_tol, rel_tol)
            
            count_steps+=1             
                
            
            if _check<1:
                _s+=1
            else:
                _s=0
            
            if _s>step_break:
                break

        return self.strategy.PDE.model.w[:]

    
class NAdamSGD:
    '''Implementation of NAdam.'''
    
    def __init__(self,PDE,beta_m=1-1e-1,beta_v=1-1e-3,epsilon=1e-8,alpha=1e-2):
        '''
        loss: the loss function
        data: the data to be used in order to minimize the loss
        beta_m: decay parameter for the average m
        beta_v: decay parameter for the average v 
        epsilon: safety parameter (to avoid division by 0)
        alpha: a learning rate that multiplies the rate of AdaDelta. 
        '''
        self.PDE=PDE

        self.beta_m=beta_m
        self.beta_v=beta_v
        self.epsilon=epsilon
        self.alpha=alpha
        
        self.steps=[]
        self.steps.append(self.PDE.model.w[:])
        self.dim=self.PDE.model.dim_w
        
        
        #The "bias corrected" m and v need beta^iteration, so I need something like this
        self.beta_m_ac=beta_m
        self.beta_v_ac=beta_v

        # counters for the decaying means of the gradient         
        self.mE=[0 for _ in self.PDE.model.w]
        self.vE=[0 for _ in self.PDE.model.w]
        
        #lists to store the changes in w         
        self.dw=[0 for _ in self.PDE.model.w]

    def update(self,abs_tol=1e-5, rel_tol=1e-3):
        '''
        update should return a number that when it is smaller than 1
        the main loop stops. Here I choose this number to be:
        sqrt(1/dim*sum_{i=0}^{dim}(grad/(abs_tol+x*rel_tol))_i^2)
        '''
        
        grad=self.PDE.randomLossGrad()#get the loss at a random point and a random boundary point            
        # accumulate the decay rates, in order to correct the averages 
        self.beta_m_ac*=self.beta_m_ac
        self.beta_v_ac*=self.beta_v_ac
        
        _w2=0
        _check=0
        for i,g in enumerate(grad):
            self.mE[i]=self.beta_m*self.mE[i] + (1-self.beta_m)*g 
            self.vE[i]=self.beta_v*self.vE[i] + (1-self.beta_v)*g**2

            self.dw[i]=self.alpha/(np.sqrt(self.vE[i]/(1-self.beta_v_ac)) + self.epsilon)
            self.dw[i]*=(self.beta_m*self.mE[i] + (1-self.beta_m)*g)/(1-self.beta_m_ac)
            self.PDE.model.w[i]=self.PDE.model.w[i] - self.dw[i]
            
            _w2=abs_tol + self.PDE.model.w[i] * rel_tol
            _check+=(g/_w2)*(g/_w2)

        _check=np.sqrt(1./self.dim *_check)
        
        self.steps.append(self.PDE.model.w[:])
        
        return _check

In [22]:
strategy=NAdamSGD(PDE,beta_m=1-1e-1,beta_v=1-1e-3,epsilon=1e-8,alpha=1e-2)
SGD=StochasticGradientDescent(strategy)

In [23]:
SGD.run(abs_tol=1e-5, rel_tol=1e-3, step_break=150,max_step=50000),len(strategy.steps)

([0.49964216512503046, 0.09999914712193492, 3.137527006000515], 5245)

In [24]:
#As you can see
print('I got w=',guess.w)
print('I expected w=',[1/2.,1/10.,np.pi])

I got w= [0.49964216512503046, 0.09999914712193492, 3.137527006000515]
I expected w= [0.5, 0.1, 3.141592653589793]


As we've already said, the solution is given for $w_0=\dfrac{1}{2},\; w_1=\dfrac{1}{10},\; w_2=\pi$.
That is, the thing works!